In [4]:
import pandas as pd
import numpy as np
import json
import datetime
from pandas.io.json import json_normalize
import scipy as sp
import scipy.fftpack
import matplotlib.pyplot as plt
import librosa
import librosa.display
from scipy.io import wavfile
from scipy.fft import fft, ifft, fftfreq
from scipy import signal
import os

In [2]:
def json_to_df():
    #load json file
    with open('S04.json','r') as f:
        data = json.loads(f.read()) 
    df = pd.json_normalize(data)

    #format timestamps as datetime objects
    df['start_time.U01'] = pd.to_datetime(df['start_time.U01']) 
    df['end_time.U01'] = pd.to_datetime(df['end_time.U01']) 

    #filter to specific 10 minute part
    part_start_time_U01 = pd.to_datetime('01:29:59')
    part_end_time_U01 = pd.to_datetime('01:39:57')
    df_new = df.loc[(df['start_time.U01'] > part_start_time_U01) & (df['start_time.U01'] < part_end_time_U01)]

    # final dataframe
    df_new = df_new.filter(['words','speaker', 'session_id','start_time.U01', 'end_time.U01'], axis=1)
    return df_new
    
df = json_to_df()
df.head()

,words,speaker,session_id,start_time.U01,end_time.U01
3177,It's basically to the end of your passport alm...,P10,S04,2021-10-01 01:29:59.910,2021-10-01 01:30:01.840
3178,"Short of like, a half a year.",P10,S04,2021-10-01 01:30:02.240,2021-10-01 01:30:03.740
3179,Maybe like five or six years from now?,P11,S04,2021-10-01 01:30:04.680,2021-10-01 01:30:06.000
3180,"Then yeah, it's worth it.",P10,S04,2021-10-01 01:30:05.940,2021-10-01 01:30:07.060
3181,"Like, if your passports expire the next two ye...",P10,S04,2021-10-01 01:30:07.060,2021-10-01 01:30:09.610


In [3]:
# make array with half a second intervals
part_start_time =  pd.to_datetime('01:29:59')
part_end_time = pd.to_datetime('01:39:57')
half_a_second = datetime.timedelta(seconds=0.5)
interval_array = []

x = part_start_time 
while x <= part_end_time:
    interval_array.append(x)
    x = x + half_a_second 


print(interval_array)



[Timestamp('2021-10-01 01:29:59'), Timestamp('2021-10-01 01:29:59.500000'), Timestamp('2021-10-01 01:30:00'), Timestamp('2021-10-01 01:30:00.500000'), Timestamp('2021-10-01 01:30:01'), Timestamp('2021-10-01 01:30:01.500000'), Timestamp('2021-10-01 01:30:02'), Timestamp('2021-10-01 01:30:02.500000'), Timestamp('2021-10-01 01:30:03'), Timestamp('2021-10-01 01:30:03.500000'), Timestamp('2021-10-01 01:30:04'), Timestamp('2021-10-01 01:30:04.500000'), Timestamp('2021-10-01 01:30:05'), Timestamp('2021-10-01 01:30:05.500000'), Timestamp('2021-10-01 01:30:06'), Timestamp('2021-10-01 01:30:06.500000'), Timestamp('2021-10-01 01:30:07'), Timestamp('2021-10-01 01:30:07.500000'), Timestamp('2021-10-01 01:30:08'), Timestamp('2021-10-01 01:30:08.500000'), Timestamp('2021-10-01 01:30:09'), Timestamp('2021-10-01 01:30:09.500000'), Timestamp('2021-10-01 01:30:10'), Timestamp('2021-10-01 01:30:10.500000'), Timestamp('2021-10-01 01:30:11'), Timestamp('2021-10-01 01:30:11.500000'), Timestamp('2021-10-01 01

In [4]:
# make arrays for where lines start/end timestamps
start_times_array = df['start_time.U01'].values
end_times_array = df['end_time.U01'].values
print(type(start_times_array))
print(type(end_times_array))


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [5]:
#clunky code, will fix if works
df_is_voice = pd.DataFrame(columns=['interval','isSpeaking'])


for i in interval_array:
    for st in start_times_array:
        voice = False
        if i >= st:
#          check end time that matches the start time of same line by index
            st_index = list(start_times_array).index(st) 
#         look if the matching end time to the start time is correct
            if end_times_array[st_index] >= i:    
                voice = True
                interval_row = pd.DataFrame([[i, voice]], columns = ['interval','isSpeaking'])
                break
    if voice != True:
        interval_row = pd.DataFrame([[i, voice]], columns = ['interval','isSpeaking'])
    df_is_voice = df_is_voice.append(interval_row)
        
        
        
print(df_is_voice.head(10))


                 interval isSpeaking
0 2021-10-01 01:29:59.000      False
0 2021-10-01 01:29:59.500      False
0 2021-10-01 01:30:00.000       True
0 2021-10-01 01:30:00.500       True
0 2021-10-01 01:30:01.000       True
0 2021-10-01 01:30:01.500       True
0 2021-10-01 01:30:02.000      False
0 2021-10-01 01:30:02.500       True
0 2021-10-01 01:30:03.000       True
0 2021-10-01 01:30:03.500       True


In [6]:
negativedf = pd.read_csv('negativedatalabels.csv')
neg_df_new = negativedf.filter(['filename','category', 'isSpeaking'], axis=1)
bigdf = [df_is_voice, neg_df_new]
result = pd.concat(bigdf)
print(result)

                    interval isSpeaking           filename        category
0    2021-10-01 01:29:59.000      False                NaN             NaN
0    2021-10-01 01:29:59.500      False                NaN             NaN
0    2021-10-01 01:30:00.000       True                NaN             NaN
0    2021-10-01 01:30:00.500       True                NaN             NaN
0    2021-10-01 01:30:01.000       True                NaN             NaN
...                      ...        ...                ...             ...
1995                     NaT      False   5-263831-B-6.wav             hen
1996                     NaT      False  5-263902-A-36.wav  vacuum_cleaner
1997                     NaT      False   5-51149-A-25.wav       footsteps
1998                     NaT      False    5-61635-A-8.wav           sheep
1999                     NaT      False     5-9032-A-0.wav             dog

[3197 rows x 4 columns]


In [7]:
bigdf = [df_is_voice, neg_df_new]
result = pd.concat(bigdf)

In [8]:
print(result)

                    interval isSpeaking           filename        category
0    2021-10-01 01:29:59.000      False                NaN             NaN
0    2021-10-01 01:29:59.500      False                NaN             NaN
0    2021-10-01 01:30:00.000       True                NaN             NaN
0    2021-10-01 01:30:00.500       True                NaN             NaN
0    2021-10-01 01:30:01.000       True                NaN             NaN
...                      ...        ...                ...             ...
1995                     NaT      False   5-263831-B-6.wav             hen
1996                     NaT      False  5-263902-A-36.wav  vacuum_cleaner
1997                     NaT      False   5-51149-A-25.wav       footsteps
1998                     NaT      False    5-61635-A-8.wav           sheep
1999                     NaT      False     5-9032-A-0.wav             dog

[3197 rows x 4 columns]


In [5]:
negativedf = pd.read_csv('negativedatalabels.csv')

In [6]:
print(negativedf)

               filename  fold  target        category  esc10  src_file take  \
0      1-100032-A-0.wav     1       0             dog   True    100032    A   
1     1-100038-A-14.wav     1      14  chirping_birds  False    100038    A   
2     1-100210-A-36.wav     1      36  vacuum_cleaner  False    100210    A   
3     1-100210-B-36.wav     1      36  vacuum_cleaner  False    100210    B   
4     1-101296-A-19.wav     1      19    thunderstorm  False    101296    A   
...                 ...   ...     ...             ...    ...       ...  ...   
1995   5-263831-B-6.wav     5       6             hen  False    263831    B   
1996  5-263902-A-36.wav     5      36  vacuum_cleaner  False    263902    A   
1997   5-51149-A-25.wav     5      25       footsteps  False     51149    A   
1998    5-61635-A-8.wav     5       8           sheep  False     61635    A   
1999     5-9032-A-0.wav     5       0             dog   True      9032    A   

      isSpeaking  
0          False  
1          Fa